# 비선형 최소 자승법 (nonlinear least squares problem)

### nonlinear optimization과의 관계
- 이전에 비선형 최적화 문제들이 주어진 함수 f : R^n -> R가 x_0에서부터 시작하여,  
  f(x_0) > f(x_1) > f(x_2) ...  
  를 만족하는 파라미터 x를 찾아가는 방법이었다.
- 이제 다루고자 하는 문제는 f : R^n -> R^m을 만족하는 다변수 함수에 대해서  
  f(x_0) > f(x_1) > f(x_2) ...  -> q = (q1, q2,..., qm)^T  
  를 만족하는 파라미터를 찾는 문제이다.
- f(x) = q를 만족하는 x를 만족하는 문제는 f_hat(x) = f(x) - q = 0을 만족하는 문제로 생각할 수 있다.  
  이 문제는 ||f_hat(x)|| = 0에 가까워지는 문제라고 볼 수 있으며, 이 문제는 미분식의 간단함을 위해서  0.5 ||f_hat(x)||^2라는  
  문제로 변형되어 사용된다.
- 다시 정리하면 아래와 같다.  
  F(x) = 0.5 ∑^m (f_i(x))^2 = 0.5 ||f(x)||^2 = 0.5 f(x)^T*f(x) 이고  
  F(x)가 최소인 x*의 값을 구하는 방법으로 비선형 최소자승법이라고 한다.  
  ※ 여기서 m은 보통 샘플의 수를 나타낸다.

## Gauss Newton 탐색방법

### 특징
- Levenberg marquardt 탐색 방법의 기본방법이고, 충분히 작은 ||h||에 대하여  
  l(h) = f(x + h) = f(x) + J(x)h 라는 근사식에서 시작한다.

### 유도
- F(x + h)의 근사식은 다음과 같다.  
  F(x + h) = L(h) = 0.5 l(h)^T l(h)
  = 0.5 * f^t(x) * f(x) + h^t * J^t(x) * f(x) + 0.5 * h^t * J^t(x) * J(x) * h  
  = F(x) + h^t * J^t(x) * f(x) + 0.5 * h^t * J^t(x) * J(x) * h
- L(h)가 최소가 되는 h를 찾으면,  
  L'(h) = J^t(x) * f(x) + J^t(x) * J(x) * h  
  L''(h) = J^t(x) * J(x) 이고
- 최소값을 가지는 L'(h_gn) = 0에서  
  J^t(x) * J(x) * h_gn = - J^t(x) * f(x)  
  이다.
- 만약 J(x)의 열벡터가 일차 독립이면, L''(h) = J^t(x)J(x)는 양의 확정행렬이고, 전역 최소값이 존재한다.
- 여기서 구해진 h_gn 은 하강 방향벡터이자. L(h)의 유일한 전역 최소자이다.

### 업데이트 방법
1. J^t(x) * J(x) * h_gn = - J^t(x) * f(x)에서 h_gn을 구한다.
2. x_{k+1} = x_k + a*h_gn 로 업데이트한다.  
   ※ a는 step size로 직선탐색 알고리즘을 이용한다.

### 예제 결과
1. 예제 1과 같이 F(x, y)가 컨벡스 함수가 아니라면 잘 수렴하지 못한다. 이것은 Levenberg maquardt 탐색방법도 비슷하다.
2. 예제 2와 같이 F(x, y)가 컨벡스 함수면 빠르게 수렴함.

In [33]:
import numpy as np
# 예제 1 가상의 a,b를 추론해보자. 
## y = a * cos(b*x) + b sin (a*x)

sample_size = 100
sample_x = 2* np.pi * np.random.rand(sample_size, 1)
noise = 0#0.01 * np.random.rand(sample_size, 1)

a = 100
b = 102

sample_y = a * np.cos( b * sample_x) + b * np.sin(a * sample_x)
noised_sample_y = sample_y + noise

iterations = 1000
init_a = 90
init_b = 102.6
step_size = 0.1
def f(x, arg_a, arg_b):
    return arg_a * np.cos( arg_b * x) + arg_b * np.sin(arg_a * x)

def gradient_f(x, arg_a, arg_b):
    gradient_a = np.cos( arg_b * x) + arg_b * np.cos(arg_a * x) * x
    gradient_b = -1.0 * arg_a * np.sin( arg_b * x) * x + np.sin(arg_a * x)
    return np.hstack((gradient_a, gradient_b))

def F(x, y, arg_a, arg_b):
    residual = y - f(x, arg_a, arg_b)
    return (0.5 * residual.T * residual)[0, 0]

updated_a = init_a
updated_b = init_b
for iter in range(iterations):
    jacobian = gradient_f(sample_x, updated_a, updated_b)
    residual = noised_sample_y - f(sample_x, updated_a, updated_b)
    j_t_j = jacobian.T @ jacobian
    minus_j_t_f = jacobian.T @ residual
    cost = F(sample_x, noised_sample_y, updated_a, updated_b)
    if iter % 100 ==0:
        print("Iter : ",iter,", Cost : ", cost, ", Parameter a : ", updated_a, ", b : ", updated_b)
    a_b_change = np.linalg.solve(j_t_j, minus_j_t_f)
    updated_a += step_size * a_b_change[0, 0]
    updated_b += step_size * a_b_change[1, 0]

Iter :  0 , Cost :  695.82347524978 , Parameter a :  90 , b :  102.6
Iter :  100 , Cost :  747.3105744313717 , Parameter a :  90.2270317147305 , b :  102.06556613760485
Iter :  200 , Cost :  736.9266715196376 , Parameter a :  90.23799835232512 , b :  102.06642493715647
Iter :  300 , Cost :  736.4873557290357 , Parameter a :  90.23836241428043 , b :  102.06645205135348
Iter :  400 , Cost :  736.4728643918726 , Parameter a :  90.23837433815554 , b :  102.06645293795269
Iter :  500 , Cost :  736.4723898552618 , Parameter a :  90.23837472852703 , b :  102.06645296697718
Iter :  600 , Cost :  736.4723743196397 , Parameter a :  90.23837474130711 , b :  102.06645296792739
Iter :  700 , Cost :  736.4723738108963 , Parameter a :  90.23837474172551 , b :  102.0664529679585
Iter :  800 , Cost :  736.4723737946441 , Parameter a :  90.23837474173916 , b :  102.06645296795949
Iter :  900 , Cost :  736.4723737943659 , Parameter a :  90.23837474173946 , b :  102.0664529679595


In [38]:
import numpy as np
# 예제 2 가상의 a, b를 추론해보자.
## f(x) = a * x^2 + b

sample_size = 100
sample_x = np.random.rand(sample_size, 1)
noise = 0.1 * np.random.rand(sample_size, 1)

a = 100
b = 102

sample_y = a * sample_x * sample_x + b * np.ones((sample_size, 1))
noised_sample_y = sample_y + noise

iterations = 3
init_a = 90
init_b = 102.6
step_size = 1
def f(x, arg_a, arg_b):
    return arg_a * x * x + arg_b * np.ones(x.shape)

def gradient_f(x, arg_a, arg_b):
    gradient_a = x * x
    gradient_b = np.ones(x.shape)
    return np.hstack((gradient_a, gradient_b))

def F(x, y, arg_a, arg_b):
    residual = y - f(x, arg_a, arg_b)
    return (0.5 * residual.T * residual)[0, 0]

updated_a = init_a
updated_b = init_b
for iter in range(iterations):
    jacobian = gradient_f(sample_x, updated_a, updated_b)
    residual = noised_sample_y - f(sample_x, updated_a, updated_b)
    j_t_j = jacobian.T @ jacobian
    minus_j_t_f = jacobian.T @ residual
    cost = F(sample_x, noised_sample_y, updated_a, updated_b)
    if iter % 1 ==0:
        print("Iter : ",iter,", Cost : ", cost, ", Parameter a : ", updated_a, ", b : ", updated_b)
    a_b_change = np.linalg.solve(j_t_j, minus_j_t_f)
    updated_a += step_size * a_b_change[0, 0]
    updated_b += step_size * a_b_change[1, 0]

Iter :  0 , Cost :  32.78390600303072 , Parameter a :  90 , b :  102.6
Iter :  1 , Cost :  0.00023782741480193515 , Parameter a :  100.00396742995875 , b :  102.04664472714445
Iter :  2 , Cost :  0.00023782741480193515 , Parameter a :  100.00396742995875 , b :  102.04664472714445
